## Install Selenium

In [248]:
!pip install selenium

## Download Chromedriver
https://selenium-python.readthedocs.io/installation.html#drivers

## Folder location
Where you save the csv file after web scraping

In [249]:
path_dir: str = r"C:\Users\thy10\Desktop\AREIX\MPF Data\Allianz try"  

## Fund list

In [250]:
fund_list=['Allianz Asian Fund - Class B (HKD)', 'Allianz Balanced Fund - Class B (HKD)', 'Allianz Capital Stable Fund - Class B (HKD)', 'Allianz Flexi Balanced Fund - Class B (HKD)', 'Allianz Greater China Fund - Class B (HKD)', 'Allianz Growth Fund - Class B (HKD)', 'Allianz Hong Kong Fund - Class B (HKD)', 'Allianz MPF Age 65 Plus Fund - Class B (HKD)', 'Allianz MPF Conservative Fund - Class B (HKD)', 'Allianz MPF Core Accumulation Fund - Class B (HKD)', 'Allianz Oriental Pacific Fund - Class B (HKD)', 'Allianz RMB Money Market Fund - Class B (HKD)', 'Allianz Stable Growth Fund - Class B (HKD)']
print(len(fund_list))

13


## Web Scraping

In [251]:
def web_scrap():
    import os
    from time import sleep
    from selenium import webdriver
    try:
        #delete file in the path 
        for count, filename in enumerate(os.listdir(path_dir)):  
            os.remove(path_dir+"/"+filename)
        
        check_list=[]
        options = webdriver.ChromeOptions()

        prefs = {"download.default_directory":path_dir}

        options.add_experimental_option('prefs', prefs)
        browser = webdriver.Chrome(chrome_options=options)

        browser.get('https://hk.allianzgi.com/en/retirement/products-solutions/mpf/')
        browser.maximize_window()

        #for the user agreement
        from selenium.webdriver.common.action_chains import ActionChains
        element = browser.find_element_by_xpath('//*[@id="overlay"]/div/div/div[5]/div/button')
        actions = ActionChains(browser)
        actions.move_to_element(element).perform()

        cb = browser.find_element_by_xpath('//*[@id="roleGateTnc"]')
        browser.execute_script("arguments[0].click();", cb)
        element.click()


        #for loading the funds
        from selenium.webdriver.common.keys import Keys
        html = browser.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_DOWN)

        #DOWNLOAD HISTORICAL NAV
        dl = browser.find_element_by_xpath('//*[@id="DailyPrices"]/div/div/div[1]/div[1]/div/a[2]/span[2]')
        browser.execute_script("arguments[0].click();", dl)

        for a in fund_list:

            #open drop down list for fund selecting
            dropdown=browser.find_element_by_xpath('//*[@id="DailyPrices"]/div/div/div[1]/div[2]/div[1]/div/div[1]/div/div[1]/span')
            browser.execute_script("arguments[0].click();", dropdown)

            elem = browser.find_element_by_xpath('//*[@id="DailyPrices"]/div/div/div[1]/div[2]/div[1]/div/div[1]/div/div[2]/div[2]/ul')
            all_li = elem.find_elements_by_tag_name("li")
            for li in all_li:
                if a == li.text:

                    check_list.append(li.text)
                    #reset the start date
                    browser.execute_script("arguments[0].click();", li)
                    datepicker=browser.find_element_by_xpath('//*[@id="excelStart"]').clear

                    #excel download button
                    excelbutton=browser.find_element_by_xpath('//*[@id="DailyPrices"]/div/div/div[1]/div[2]/div[1]/div/div[4]/a/span[2]')
                    browser.execute_script("arguments[0].click();", excelbutton)

        sleep(5)
        browser.quit()
    except:
        print("Error occured")
        web_scrap()

In [252]:
web_scrap()

<ipython-input-251-51d16dcbf689>:16: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options)


## Concate Fund
output as Allianz.csv

In [270]:
dfs=[]
funds_monthly_list=[]#for monthly

import pandas as pd
from datetime import datetime
for count, filename in enumerate(os.listdir(path_dir)):   

    #read as df
    filepath=path_dir+"/"+filename
    df = pd.read_excel(filepath)
    
    #get fund name
    a=df.iloc[8][0]
    part = a.partition("Fund")
    fundname=part[0]+part[1]
    #new_name_list.append(fundname)
    print(part[0]+part[1])
    
    #modify the format
    df=df.iloc[8:,[2,3]]
    df = df.dropna()
    df.columns=['Valuation Date',fundname]
    df['Valuation Date'] = pd.to_datetime(df['Valuation Date'], format="%Y-%m-%d")
    df.set_index("Valuation Date", inplace = True) 
    df.sort_values(by=['Valuation Date'], inplace=True)
    df = df.apply(pd.to_numeric, errors='ignore')
    
    #*****************************Daily*****************************
    dfs.append(df)
    #*****************************End of daily*****************************
    
    
    #*****************************Monthly*****************************
    df1=df.copy()
    df1.index=pd.to_datetime(df1.index, format="%y-%m-%d").to_period('M')

    #dataframes
    df_start=df1.groupby(df1.index).head(1)  #first one of the month
    df_end=df1.groupby(df1.index).tail(1) #last one of the month
    

    df_fund_monthly = df_start.copy()

    for i in range(len(df_start)):
        if(df_end.iloc[i][0]==df_start.iloc[i][0]and i!=0):
            monthly_return = (df_end.iloc[i][0]-df_end.iloc[i-1][0])/df_end.iloc[i-1][0]
        else:
            monthly_return = (df_end.iloc[i][0]-df_start.iloc[i][0])/df_start.iloc[i][0]    

        df_fund_monthly.iloc[i][fundname] = monthly_return
    funds_monthly_list.append(df_fund_monthly)
    #*****************************End of monthly*****************************
    
    
#print(dfs)
dfA = pd.concat(dfs,axis=1)
print(dfA)
dfA.to_csv('Allianztry.csv')

df_monthly = pd.concat(funds_monthly_list,axis=1)  
df_monthly.to_csv('Allianz_df_monthly.csv')

Allianz Balanced Fund
before is 
Allianz Balanced Fund    object
dtype: object
now is 
Allianz Balanced Fund    float64
dtype: object


Allianz Oriental Pacific Fund
before is 
Allianz Oriental Pacific Fund    object
dtype: object
now is 
Allianz Oriental Pacific Fund    float64
dtype: object


Allianz RMB Money Market Fund
before is 
Allianz RMB Money Market Fund    object
dtype: object
now is 
Allianz RMB Money Market Fund    float64
dtype: object


Allianz Stable Growth Fund
before is 
Allianz Stable Growth Fund    object
dtype: object
now is 
Allianz Stable Growth Fund    float64
dtype: object


Allianz Capital Stable Fund
before is 
Allianz Capital Stable Fund    object
dtype: object
now is 
Allianz Capital Stable Fund    float64
dtype: object


Allianz Flexi Balanced Fund
before is 
Allianz Flexi Balanced Fund    object
dtype: object
now is 
Allianz Flexi Balanced Fund    float64
dtype: object


Allianz Greater China Fund
before is 
Allianz Greater China Fund    object
dtype: obj